### Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import string
from tqdm.notebook import tqdm
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [2]:
# Create a Kaggle API client
import os
os.environ['KAGGLE_USERNAME'] = 'mustabshiribnamin'
os.environ['KAGGLE_KEY'] = '0eff4183442e547cfe97d8be3e922d42' 
import kaggle as kg
kg.api.dataset_download_files(dataset = "bhavikjikadara/fake-news-detection", path='dataset', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/fake-news-detection


### View Data

In [3]:
true_df = pd.read_csv('dataset/true.csv')
fake_df = pd.read_csv('dataset/fake.csv')

In [4]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [6]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


### Data Preprocessing

In [8]:
true_df.duplicated().sum()

206

In [9]:
true_df.drop_duplicates(inplace=True)

In [10]:
fake_df.duplicated().sum()

3

In [11]:
fake_df.drop_duplicates(inplace=True)

In [12]:
true_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [13]:
fake_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [14]:
# labeling the true and false state
true_df['label'] = 0
fake_df['label'] = 1

In [15]:
# concating true and fake data
df = pd.concat([fake_df, true_df], ignore_index=True)
# shuffling the rows of the DataFrame and resetting the index
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
#replace underscores and making title format
df.columns = df.columns.str.replace('_', ' ').str.title()

In [17]:
df.head()

,Title,Text,Subject,Date,Label
0,Daily Shooter Academy: Florida Woman Shot Dead...,21st Century Wire says This is something which...,US_News,"August 10, 2016",1
1,Turkish police arrest suspect in killing of Sy...,ISTANBUL (Reuters) - Turkish police arrested a...,worldnews,"September 30, 2017",0
2,Trump administration does not want limits on w...,WASHINGTON (Reuters) - President Donald Trump’...,politicsNews,"October 30, 2017",0
3,"CUBAN INVASION: 7,000 Crossing Border Into U.S...",Thanks to Obama s open border policy and a rec...,politics,"Jan 26, 2016",1
4,Jack Lew urges Senate to act on Puerto Rico debt,WASHINGTON (Reuters) - U.S. Treasury Secretary...,politicsNews,"June 27, 2016",0


In [18]:
# Clean text
stopword = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean(text):
    text = str(text).lower()
    # remove text within square brackets
    text = re.sub('\[.*?\]', '', text) 
    # remove http links
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # remove html tags
    text = re.sub('<.*?>+', '', text)
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # remove newline chars
    text = re.sub('\n', '', text)
    # remove all word containing numbers
    text = re.sub('\w*\d\w*', '', text)
    # remove stopwords
    text = [word for word in text.split(' ') if word not in stopword]
    # applies stemming to words
    text = [stemmer.stem(word) for word in text]
    text = " ".join(text)
    return text

df['Text'] = df['Text'].apply(lambda x: clean(x))
df['Title'] = df['Title'].apply(lambda x: clean(x))

In [19]:
df.head()

,Title,Text,Subject,Date,Label
0,daili shooter academi florida woman shot dead ...,centuri wire say someth warn year ever sinc ...,US_News,"August 10, 2016",1
1,turkish polic arrest suspect kill syrian activist,istanbul reuter turkish polic arrest suspect ...,worldnews,"September 30, 2017",0
2,trump administr want limit war author,washington reuter presid donald trump’ top na...,politicsNews,"October 30, 2017",0
3,cuban invas cross border us help fed,thank obama open border polici recent agreemen...,politics,"Jan 26, 2016",1
4,jack lew urg senat act puerto rico debt,washington reuter us treasuri secretari jack ...,politicsNews,"June 27, 2016",0


### Sentiment Analysis

In [20]:
sia = SentimentIntensityAnalyzer()
result = {}

for i, row in tqdm(df.iterrows(), total=len(df), desc="Sentiment Analysis"):
    text = row['Text']
    my_id = row['Title'] 
    result[my_id] = sia.polarity_scores(text)

Sentiment Analysis:   0%|          | 0/44689 [00:00<?, ?it/s]

In [21]:
vaders = pd.DataFrame(result).T
vaders = vaders.reset_index().rename(columns={'index': 'Title'})
vaders = vaders.merge(df, how='left')

In [22]:
vaders.head()

,Title,neg,neu,pos,compound,Text,Subject,Date,Label
0,daili shooter academi florida woman shot dead ...,0.175,0.767,0.058,-0.9633,centuri wire say someth warn year ever sinc ...,US_News,"August 10, 2016",1
1,daili shooter academi florida woman shot dead ...,0.175,0.767,0.058,-0.9633,centuri wire say someth warn year ever sinc ...,Middle-east,"August 10, 2016",1
2,turkish polic arrest suspect kill syrian activist,0.214,0.786,0.000,-0.9686,istanbul reuter turkish polic arrest suspect ...,worldnews,"September 30, 2017",0
3,trump administr want limit war author,0.126,0.816,0.057,-0.9882,washington reuter presid donald trump’ top na...,politicsNews,"October 30, 2017",0
4,cuban invas cross border us help fed,0.024,0.834,0.143,0.9886,thank obama open border polici recent agreemen...,politics,"Jan 26, 2016",1


### Model Training

In [23]:
from imblearn.over_sampling import RandomOverSampler
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix


In [24]:
df1 = vaders.copy()

In [25]:
df1['content'] = df1['Title'] + ' ' + df1['Text']

In [26]:
X = df1['content']
y = df1['Label']

In [27]:
# Scaling "Label" oversampling for balanced class distribution
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(df1['content'].values.reshape(-1, 1), df1['Label'])
X_res = X_res.flatten()

In [28]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='OOV')
tokenizer.fit_on_texts(X_res)
sequences = tokenizer.texts_to_sequences(X_res)
padded = pad_sequences(sequences, maxlen=1000)

In [29]:
# Save the tokenizer
with open('Models/tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

In [30]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded, y_res, test_size=0.2, random_state=42)

In [31]:
model = keras.Sequential()

# converts integer-encoded word indices (from the tokenized text) into dense vectors of fixed size
model.add(keras.layers.Embedding(input_dim=10000, output_dim=32))  
model.add(keras.layers.SimpleRNN(32, activation='relu', dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model_checkpoint = keras.callbacks.ModelCheckpoint('model.{epoch:02d}-{val_loss:.2f}.keras', monitor='val_loss', save_best_only=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 114s 96ms/step - accuracy: 0.8512 - loss: 0.3651 - val_accuracy: 0.9142 - val_loss: 0.2124
Epoch 2/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 152s 129ms/step - accuracy: 0.9604 - loss: 0.3418 - val_accuracy: 0.9035 - val_loss: 0.2274
Epoch 3/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 145s 123ms/step - accuracy: 0.9693 - loss: 0.0903 - val_accuracy: 0.9045 - val_loss: 0.2310


In [33]:
evaluation_result = model.evaluate(X_test, y_test)

print("Test Loss:", evaluation_result[0])
print("Test Accuracy:", evaluation_result[1])

294/294 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9136 - loss: 0.2117
Test Loss: 0.21238742768764496
Test Accuracy: 0.9141823053359985


In [34]:
model.summary()
model.save('simple_rnn_model.keras')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1000, 32)       │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 966,341 (3.69 MB)

 Trainable params: 322,113 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 644,228 (2.46 MB)

In [35]:
model_lstm = keras.Sequential()
model_lstm.add(keras.layers.Embedding(input_dim=10000, output_dim=32))
model_lstm.add(keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True)))
model_lstm.add(keras.layers.Dropout(0.5))
model_lstm.add(keras.layers.LSTM(32))
model_lstm.add(keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.01)))
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lstm.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model_lstm_checkpoint = keras.callbacks.ModelCheckpoint('model_lstm.{epoch:02d}-{val_loss:.2f}.keras', monitor='val_loss', save_best_only=True)

history = model_lstm.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 484s 409ms/step - accuracy: 0.9288 - loss: 0.2013 - val_accuracy: 0.9932 - val_loss: 0.0435
Epoch 2/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 481s 410ms/step - accuracy: 0.9929 - loss: 0.0423 - val_accuracy: 0.9753 - val_loss: 0.1008
Epoch 3/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 296s 252ms/step - accuracy: 0.9851 - loss: 0.0641 - val_accuracy: 0.9855 - val_loss: 0.0596


In [37]:
evaluation_result = model_lstm.evaluate(X_test, y_test)

print("Test Loss:", evaluation_result[0])
print("Test Accuracy:", evaluation_result[1])

294/294 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - accuracy: 0.9933 - loss: 0.0425
Test Loss: 0.04351022467017174
Test Accuracy: 0.9931856989860535


In [38]:
model_lstm.summary()
model_lstm.save('lstm_model.keras')

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 1000, 32)       │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 1000, 64)       │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1000, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,047,269 (4.00 MB)

 Trainable params: 349,089 (1.33 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 698,180 (2.66 MB)

In [39]:
new_model = keras.models.load_model('lstm_model.keras')
predictions = new_model.predict(X_test)

294/294 ━━━━━━━━━━━━━━━━━━━━ 17s 56ms/step
